# Ejemplo 2: Localización de antenas telefónicas

Diego R. Páez Ardila

**Contexto**: Un problema muy usual en el área de negocios es el de la localización de fabricas, almacenes,  centros de distribución y torres de transmisión telefónica. En esos problemas el objetivo y minimizar la distancia total entre los centros consumidores y el centro de distribución, reduciendo así, teóricamente el costo de transporte o perdidas por transmisión. 

Lo usual es ubicarse en un eje cartesiano sobre un mapa y determinar la posición de los centros de consumidores en relación a un punto de origen aleatorio.

**Problema**: El gerente de proyectos de una empresa de telefonía celular. Tiene que localizar una antena de retransmisión para atender a tres localidades del área metropolitana. Por problemas técnicos la antena no puede estar a mas de 10 km del centro de cada ciudad. Considerando las ubicaciones de las ciudades, termine el mejor lugar para ubicar la torre.

**Variables del problema**

* $X = $ Coordenada en el eje X de torre de transmisión;
* $Y = $  Coordenada en el eje Y de torre de transmisión;

**Objetivo**

Minimizar la distancia total entre los centros consumidores y el centro de distribución.

<img src="ubicacion_ciudades.png" width="700">

**Función objetivo y Restricciones**

Se utiliza la sumatoria de la distancia euclidiana existente entre cada ciudad y la antena de transmisión. Adicionalmente se incluyen las restricciones de que la antena no puede estar a mas de 10 km de cada ciudad.

<img src="ecuaciones.png" width="700">


## Instalación de Paquetes

In [56]:
# https://deap.readthedocs.io/en/master/
!pip install deap

In [57]:
# Librerías
import random
import numpy as np
from deap import algorithms, base, creator, tools

In [58]:
#Datos con la localización de las ciudades
x_location = [-5, 2, 10]
y_location = [10, 1, 5]

## Preparación para Optimización

1. **Función objetivo**:
2. **Variables de decisión**
3. **Operadores**

In [59]:
"""
Función Objetivo
"""
def objective_function(individual):
  """
  Sumatoria de la distancia euclidiana entre cada ciudad y la antena

  Args:
      individual (List): Lista de individuos generados por el AG

  Returns:
      Tuple: Sumatoria de las distancias calculadas
  """
  distancia_1 = np.sqrt((x_location[0]-individual[0])**2 + (y_location[0]-individual[1])**2)
  distancia_2 = np.sqrt((x_location[1]-individual[0])**2 + (y_location[1]-individual[1])**2)
  distancia_3 = np.sqrt((x_location[2]-individual[0])**2 + (y_location[2]-individual[1])**2)
  return (distancia_1 + distancia_2 + distancia_3),

In [60]:
# https://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html
def feasible(individual):
  """
  Viabilidad del indivudo generado por el GA

  Args:
      individual (List): Lista de individuos

  Returns:
      Bool: True si es viable, 
            False si no es viable
  """
  
  distancia_1 = np.sqrt((x_location[0]-individual[0])**2 + (y_location[0]-individual[1])**2)
  distancia_2 = np.sqrt((x_location[1]-individual[0])**2 + (y_location[1]-individual[1])**2)
  distancia_3 = np.sqrt((x_location[2]-individual[0])**2 + (y_location[2]-individual[1])**2)
  
  if distancia_1 > 10:
    return False
  if distancia_2 > 10:
    return False
  if distancia_3 > 10:
    return False
  return True

In [61]:
# Creación estructura de fitness e individuo
##para problema de maximización el peso es positivo y minimización el peso es negativo
creator.create("FitnessMax", base.Fitness, weights=(-1.0,)) 
creator.create("Individual", list, fitness=creator.FitnessMax)

In [62]:
toolbox = base.Toolbox()

# Generador de atributos reales: nombre, función que genera cada variable, intervalo (limite superior e inferior)
toolbox.register("attr_real", random.uniform, -20, 20)

# Generador de individuo
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_real, 2)

# Generar la población

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

:key: **Nota**: la función `sys.float_info.max` retorna el valor máximo que puede tomar un número de punto flotante en el sistema. En este caso se utiliza para definir el tamaño de la penalización.

In [63]:
import sys
#Inicializar Operadores: https://deap.readthedocs.io/en/master/api/tools.html
toolbox.register("evaluate",objective_function)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, sys.float_info.max)) # sys.float_info_max
toolbox.register("mate", tools.cxSimulatedBinaryBounded, eta=0.5, low=-20, up=20)
toolbox.register('mutate', tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select",tools.selTournament, tournsize=3)

In [64]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind:ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

## Ciclo Evolutivo

In [65]:
pop, log =algorithms.eaSimple(population=pop, 
                              toolbox=toolbox, 
                              cxpb=0.5, 
                              mutpb=0.1, 
                              ngen=20, 
                              stats=stats, 
                              halloffame=hof, 
                              verbose=True)

gen	nevals	avg	std	min    	max         
0  	100   	inf	inf	20.1442	1.79769e+308
1  	54    	inf	inf	20.1442	1.79769e+308
2  	42    	inf	inf	20.1442	1.79769e+308
3  	51    	inf	inf	20.1442	1.79769e+308
4  	49    	inf	inf	20.1442	1.79769e+308
5  	54    	inf	inf	20.1442	1.79769e+308
6  	50    	inf	inf	20.1442	1.79769e+308
7  	49    	inf	inf	20.1442	1.79769e+308
8  	66    	20.1442	0  	20.1442	20.1442     
9  	47    	1.79769e+306	inf	20.1442	1.79769e+308
10 	57    	20.1442     	0  	20.1442	20.1442     
11 	52    	20.1442     	0  	20.1442	20.1442     
12 	49    	1.79769e+306	inf	20.1442	1.79769e+308
13 	62    	1.79769e+306	inf	20.1442	1.79769e+308
14 	41    	20.1442     	0  	20.1442	20.1442     
15 	63    	1.79769e+306	inf	20.1442	1.79769e+308
16 	58    	20.1458     	0.0135442	20.1442	20.2777     
17 	68    	20.1442     	0.000340633	20.1442	20.1476     
18 	46    	20.1442     	0          	20.1442	20.1442     
19 	56    	1.79769e+306	inf        	20.1442	1.79769e+308
20 	70    	1.79769e+306	inf

# Resultados

In [66]:
# Melhor solução
print("Mejor Individuo - Localización de la Antena: ")
print(hof[0])
X_ant = hof[0][0]
Y_ant = hof[0][1]

distancia_1 = np.sqrt((x_location[0]-X_ant)**2 + (y_location[0]-Y_ant)**2)
distancia_2 = np.sqrt((x_location[1]-X_ant)**2 + (y_location[1]-Y_ant)**2)
distancia_3 = np.sqrt((x_location[2]-X_ant)**2 + (y_location[2]-Y_ant)**2)

print("Distancia Antena - Bucaramanga: " + str(round(distancia_1,2)))
print("Distancia Antena - Girón: " + str(round(distancia_2,2)))
print("Distancia Antena - Floridablanca: " + str(round(distancia_3,2)))

#validación del mejor individuo
print("Función Objetivo:")
print(objective_function(hof[0]))

Mejor Individuo - Localización de la Antena: 
[2.5497603954530668, 3.6566342324383427]
Distancia Antena - Bucaramanga: 9.86
Distancia Antena - Girón: 2.71
Distancia Antena - Floridablanca: 7.57
Función Objetivo:
(20.14419548800391,)
